In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers, activations, \
    optimizers, utils, losses, initializers, metrics, callbacks
print(tf.config.list_physical_devices('GPU'))
tf.config.list_physical_devices()

In [ ]:
epochs = 100
batch_size = 32
patience = 5
learning_rate = 0.001
model_path = './model/sigmoid.keras'

In [ ]:
model = models.Sequential([
    layers.Resizing(128, 128),
    layers.Rescaling(1.0/255),
    layers.BatchNormalization(),

    layers.Conv2D(32, (5, 5),
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(32, (3, 3),
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    
    layers.Dropout(0.5),
    layers.Dense(128,
        activation = 'tanh',
        kernel_initializer = initializers.RandomNormal()
    ),

    layers.Dense(32,
        activation = 'tanh',
        kernel_initializer = initializers.RandomNormal()
    ),
    
    layers.Dense(29,
        activation = 'sigmoid',
        kernel_initializer = initializers.RandomNormal()
    )
])

In [ ]:
model.compile(
    optimizer = optimizers.Adam(
        learning_rate = learning_rate
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [ metrics.SparseCategoricalAccuracy() ]
)

In [ ]:
train = utils.image_dataset_from_directory(
    'asl_alphabet_train',
    validation_split = 0.2,
    subset = 'training',
    seed = 123,
    shuffle = True,
    image_size = (512, 512),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    'asl_alphabet_train',
    validation_split = 0.2,
    subset = 'validation',
    seed = 123,
    shuffle = True,
    image_size = (512, 512),
    batch_size = batch_size
)
train.class_names

In [ ]:
model.fit(train,
    epochs = epochs,
    validation_data = test,
    callbacks= [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model = models.load_model('./model/sigmoid.keras')
model.summary()

In [ ]:
def load_img(img_path):
    img = Image.open(img_path)
    img_array = np.asarray(img)
    return np.expand_dims(img_array, axis=0)

In [ ]:
np.argmax(model.predict(load_img("asl_alphabet_train\\s\\0a6fc47a-3260-40c7-b59e-dcef24cb0e91.rgb_0000.png")))

In [ ]:
np.argmax(model.predict(load_img("foto1.jpg")))